In [2]:
#test common_voice test data

import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re
import pykakasi 
import MeCab


wer = load_metric("wer")
cer = load_metric("cer")

model = Wav2Vec2ForCTC.from_pretrained("ttop324/wav2vec2-live-japanese").to("cuda")
processor = Wav2Vec2Processor.from_pretrained("ttop324/wav2vec2-live-japanese")
test_dataset = load_dataset("common_voice", "ja", split="test")


chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\‘\”\�‘、。．！，・―─~｢｣『』\\\\※\[\]\{\}「」〇？…]'
wakati = MeCab.Tagger("-Owakati")
kakasi = pykakasi.kakasi()
kakasi.setMode("J","H")      # kanji to hiragana
kakasi.setMode("K","H")      # katakana to hiragana
conv = kakasi.getConverter()


FULLWIDTH_TO_HALFWIDTH = str.maketrans(
    '　０１２３４５６７８９ａｂｃｄｅｆｇｈｉｊｋｌｍｎｏｐｑｒｓｔｕｖｗｘｙｚＡＢＣＤＥＦＧＨＩＪＫＬＭＮＯＰＱＲＳＴＵＶＷＸＹＺ！゛＃＄％＆（）＊＋、ー。／：；〈＝〉？＠［］＾＿‘｛｜｝～',
    ' 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&()*+,-./:;<=>?@[]^_`{|}~',
)
def fullwidth_to_halfwidth(s):
    return s.translate(FULLWIDTH_TO_HALFWIDTH)


def preprocessData(batch):
    batch["sentence"] = fullwidth_to_halfwidth(batch["sentence"])
    batch["sentence"] = re.sub(chars_to_ignore_regex,' ', batch["sentence"]).lower()  #remove special char
    batch["sentence"] = wakati.parse(batch["sentence"])                              #add space
    batch["sentence"] = conv.do(batch["sentence"])                                   #covert to hiragana
    batch["sentence"] = " ".join(batch["sentence"].split())+" "                         #remove multiple space 
    
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = torchaudio.functional.resample(speech_array, sampling_rate, 16000)[0].numpy()    
    return batch


test_dataset = test_dataset.map(preprocessData)



# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
	inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

	with torch.no_grad():
		logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits

	pred_ids = torch.argmax(logits, dim=-1)
	batch["pred_strings"] = processor.batch_decode(pred_ids)
	return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)

print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["sentence"])))
print("CER: {:2f}".format(100 * cer.compute(predictions=result["pred_strings"], references=result["sentence"])))






Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Reusing dataset common_voice (/home/user/.cache/huggingface/datasets/common_voice/ja/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b)
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated method setMode. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated method getConverter. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.
Parameter 'function'=<function preprocessData at 0x7f2b4404d9e0> of the transform datasets.arrow_da

  0%|          | 0/632 [00:00<?, ?ex/s]

/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/ipykernel_launcher.py:40: DeprecationWarning: Call to deprecated method do. (Old API will be removed in v3.0.) -- Deprecated since version 2.1.


  0%|          | 0/79 [00:00<?, ?ba/s]

/home/user/anaconda3/envs/tf2/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1631630836880/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


WER: 21.485999
CER: 9.829138


In [4]:
#usage
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor


model = Wav2Vec2ForCTC.from_pretrained("wav2vec2_large_xlsr_japanese_hiragana")
processor = Wav2Vec2Processor.from_pretrained("wav2vec2_large_xlsr_japanese_hiragana/")
test_dataset = load_dataset("common_voice", "ja", split="test")



# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = torchaudio.functional.resample(speech_array, sampling_rate, 16000)[0].numpy()    
    return batch


test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset[:2]["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
	logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset[:2]["sentence"])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Reusing dataset common_voice (/home/user/.cache/huggingface/datasets/common_voice/ja/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b)
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/common_voice/ja/6.1.0/078d412587e9efeb0ae2e574da99c31e18844c496008d53dc5c60f4159ed639b/cache-829052a33422b2f7.arrow


Prediction: ['そぼ を おもにきねん を くさい くろころがし て い', 'さいふ を な くし かん の で こうばん で いき ます']
Reference: ['祖母は、おおむね機嫌よく、サイコロをころがしている。', '財布をなくしたので、交番へ行きます。']
